# Data generation for service evaluation

In [ ]:
import numpy as np
import pandas as pd

np.random.seed()
N = 1000   # number of synthetic ship logs

# ============================
# Domain 1: Route & Navigation
# ============================
route_nav = pd.DataFrame({
    'distance_nm': np.random.uniform(50, 5000, N),         # nautical miles
    'planned_speed_kn': np.random.uniform(10, 25, N),      # knots
    'actual_speed_kn': np.random.uniform(8, 26, N),
    'eta_hours': np.random.uniform(5, 500, N),
    'route_risk_score': np.random.uniform(0, 1, N),        # piracy/weather risk
    'fuel_capacity_tons': np.random.uniform(50, 300, N),
    'fuel_used_tons': np.random.uniform(20, 290, N),
})

route_nav['speed_variance'] = (
    route_nav['planned_speed_kn'] - route_nav['actual_speed_kn']
)

# ============================
# Domain 2: Weather Conditions
# ============================
weather = pd.DataFrame({
    'wave_height_m': np.random.uniform(0, 12, N),
    'wind_speed_kn': np.random.uniform(0, 60, N),
    'wind_direction_deg': np.random.uniform(0, 360, N),
    'visibility_km': np.random.uniform(1, 20, N),
    'storm_probability': np.random.uniform(0, 1, N),
    'precipitation_mm': np.random.uniform(0, 100, N),
})

# ============================
# Domain 3: Vessel Load & Capacity
# ============================
vessel_load = pd.DataFrame({
    'max_capacity_tons': np.random.uniform(1000, 30000, N),
    'current_load_tons': np.random.uniform(200, 29000, N),
    'num_containers': np.random.randint(50, 2000, N),
    'reefer_containers': np.random.randint(0, 300, N),
    'bulk_cargo_tons': np.random.uniform(0, 5000, N),
    'ballast_water_tons': np.random.uniform(0, 10000, N),
})

vessel_load['load_ratio'] = (
    vessel_load['current_load_tons'] / vessel_load['max_capacity_tons']
)

# ============================
# Domain 4: Cargo Storage & Loading
# ============================
cargo_types = [
    "containers", "bulk", "liquid", "hazardous", "reefer",
    "vehicles", "general_goods"
]

storage_loading = pd.DataFrame({
    'cargo_type': np.random.choice(cargo_types, N),
    'cargo_weight_tons': np.random.uniform(1, 500, N),
    'loading_speed_tph': np.random.uniform(20, 200, N),     # tons per hour
    'unloading_speed_tph': np.random.uniform(20, 200, N),
    'storage_temp_req_C': np.random.uniform(-20, 30, N),    # some cargo needs cooling
    'containerized': np.random.choice([0, 1], N),
})

# Convert cargo_type categorical → numeric index
storage_loading['cargo_type_id'] = pd.factorize(storage_loading['cargo_type'])[0]

# ============================
# Domain 5: Temporal & Port Activity
# ============================
temporal_port = pd.DataFrame({
    'arrival_hour': np.random.randint(0, 24, N),
    'arrival_day': np.random.randint(0, 7, N),
    'port_congestion_level': np.random.uniform(0, 1, N),
    'docking_delay_hours': np.random.uniform(0, 48, N),
    'tugboat_availability': np.random.choice([0, 1], N),
})

# ============================
# Combine & Normalize
# ============================
datasets = [
    route_nav,
    weather,
    vessel_load,
    storage_loading.drop(columns=['cargo_type']),  # use numeric features only
    temporal_port
]

DATA_MATRIX = np.hstack([df.values for df in datasets])
DATA_MATRIX = (DATA_MATRIX - DATA_MATRIX.min(axis=0)) / (np.ptp(DATA_MATRIX, axis=0) + 1e-8)

# ============================
# Multi-node target generator
# ============================
# ============================
# Graph-based Multi-node Targets
# ============================

D_graph = 3
candidate_dims = [7, 6,6]# 6, 7, 6]  # updated target dimensions for 5 nodes

def generate_targets_per_node(DATA_MATRIX, candidate_dims, D_graph):
    targets = []
    for node_idx in range(D_graph):
        row = DATA_MATRIX[node_idx % len(DATA_MATRIX)]
        dim = candidate_dims[node_idx]
        if len(row) >= dim:
            sampled = row[:dim]
        else:
            sampled = np.pad(row, (0, dim - len(row)), constant_values=0.5)
        targets.append({'target': sampled})
    return targets

synthetic_targets = generate_targets_per_node(DATA_MATRIX, candidate_dims, D_graph)

# Test sizes
for i, t in enumerate(synthetic_targets):
    print(f"Node {i} target size: {len(t['target'])}")
candidate_dims = [[7]*10, [6]*10,[6]*10, ]#[7], [6]]  # updated target dimensions for 5 nodes


# Metrics Evaluator Using Branch and Bound

In [ ]:

import numpy as np

# ---------------- FEATURE KEYS ----------------
FEATURE_KEYS = ['Distance', 'Speed', 'Load', 'Capacity', 'TempRequirement', 'Containerization']

FEATURE_TARGET = [[1]*len(FEATURE_KEYS) for _ in range(10)]  # 10 nodes, all features active

def default_feature_values():
    return {k: 0.5 for k in FEATURE_KEYS}


METRIC_KEYS = [
    'Reliability',          # Node reliability / uptime
    'Efficiency',           # Energy conversion / utilization efficiency
    'Cost',                 # Operational cost
   # 'Flexibility',          # Ability to handle variable load / demand response
   # 'Emissions',            # Environmental impact / CO2 equivalent
   # 'RenewableUse',         # Utilization of renewable generation
   # 'VoltageStability',     # Node voltage stability
   # 'CongestionRisk',       # Risk of grid congestion
   # 'BatteryWear',          # Battery degradation proxy
   # 'DemandResponseSlack'   # Capacity left for demand response
]

METRIC_TARGET = [[1,1,1,1,1,],[1,1,1,0,0],[1,1,1,0,0]]#[[1]*len(FEATURE_KEYS) for _ in range(10)]

import numpy as np

METRIC_FORMULAS = [
    lambda x: np.exp(-((x-0.5)**2)/0.05),   # Reliability
    #lambda x: np.exp(-((x-0.6)**2)/0.08),   # Efficiency
    lambda x: 0.5 * x**2 + 0.1,             # Cost
   # lambda x: 1 / (1 + 3*x),                # Flexibility
   # lambda x: 0.7 * x**0.9,                 # Emissions
  #  lambda x: np.log1p(x),                  # Renewable utilization
#]#   lambda x: np.tanh(2*x),                 # Voltage stability
  #  lambda x: np.exp(-x),                    # Congestion risk
   # lambda x: np.sqrt(x),                    # Battery wear approximation
    lambda x: (1 - x)**2                     # Demand response slack
]


metric_feature_map = {
    # Reliability depends on stress vs capability
    'Reliability': [
        'Load',
        'Capacity',
        'TempRequirement'
    ],

    # Efficiency depends on speed under load
    'Efficiency': [
        'Speed',
        'Load',
        'Containerization'
    ],

    # Cost increases with distance, handling, and load
    'Cost': [
        'Distance',
        'Load',
        'Containerization'
    ]
}




class BranchBoundOptimizer:
    """
    Exact, non-recursive Branch & Bound optimizer for 1D continuous problems.
    """
    def __init__(
        self,
        tol=1e-3,
        max_depth=20,
        minimize=True,
        value_range=(0.0, 5.0)
    ):
        self.tol = tol
        self.max_depth = max_depth
        self.minimize = minimize
        self.value_range = value_range

    def optimize(self, features, y=None, metric_mask=None):
        y = np.zeros(3) if y is None else np.array(y[:3])
        base = np.mean(list(features.values())) + np.mean(y)

        # Initial interval (shifted by base)
        a0, b0 = self.value_range
        a0 += base
        b0 += base

        # Work list: (a, b, depth)
        work = [(a0, b0, 0)]

        best_x = None
        best_score = np.inf if self.minimize else -np.inf

        def better(s1, s2):
            return s1 < s2 if self.minimize else s1 > s2

        while work:
            a, b, depth = work.pop()

            mid = 0.5 * (a + b)

            # Evaluate midpoint
            mv = [f(mid) if m else 0.0 for f, m in zip(METRIC_FORMULAS, metric_mask)]
            score = sum(mv)

            if best_x is None or better(score, best_score):
                best_x = mid
                best_score = score

            # Termination condition
            if depth >= self.max_depth or (b - a) < self.tol:
                continue

            # Branch (NO pruning — exact)
            work.append((a, mid, depth + 1))
            work.append((mid, b, depth + 1))

        return best_x


class MetricsEvaluator:
    """Compute node metrics using feature subsets and exact B&B optimizer per metric."""
    def __init__(
        self,
        data_matrix,
        metric_formulas=METRIC_FORMULAS,
        metric_feature_map=metric_feature_map,
        feature_keys=FEATURE_KEYS,
        feature_target=None,
        metric_target=None,
        tol=1e-3,
        max_depth=20,
        minimize=False,
        value_range=(0.0, 5.0)
    ):
        self.data_matrix = data_matrix
        self.metric_formulas = metric_formulas
        self.metric_feature_map = metric_feature_map
        self.feature_keys = feature_keys
        self.feature_target = feature_target or [[1]*len(feature_keys) for _ in range(data_matrix.shape[0])]
        self.metric_target = metric_target or [[1]*len(metric_formulas) for _ in range(data_matrix.shape[0])]
        self.num_nodes = data_matrix.shape[0]

        # Create a B&B optimizer instance
        self.optimizer = BranchBoundOptimizer(
            tol=tol,
            max_depth=max_depth,
            minimize=minimize,
            value_range=value_range
        )

    def extract_features(self, node_idx):
        """Return active features for a node as {feature_key: value}."""
        row = self.data_matrix[node_idx]
        mask = self.feature_target[node_idx]
        features = {k: v for k, v, m in zip(self.feature_keys, row, mask) if m}
        return features

    def compute_node_metrics(self, node_idx, y=None):
        """
        Compute metrics for a single node using:
        - relevant features from metric_feature_map
        - B&B optimizer per metric
        """
        features = self.extract_features(node_idx)
        metric_mask = self.metric_target[node_idx]

        metric_values = {}

        for key, formula, mask in zip(METRIC_KEYS, self.metric_formulas, metric_mask):
            if mask:
                # Pick only the relevant features for this metric
                relevant_features = [features[f] for f in self.metric_feature_map[key] if f in features]
                x = np.mean(relevant_features) if relevant_features else 0.0

                # Optimize the scalar using B&B
                opt_value = self.optimizer.optimize(features={key: x}, y=y, metric_mask=[1])
                metric_values[key] = formula(opt_value)
            else:
                metric_values[key] = 0.0

        metric_values['score'] = sum(metric_values.values())
        return metric_values


In [ ]:
METRIC_TARGET

In [ ]:
synthetic_targets

# Fuzzy Hierarchical Multiplex Model

In [ ]:


import numpy as np
import matplotlib.pyplot as plt
import networkx as nx


# ---------------- CONFI

outer_generations = 1000
outer_cost_limit = 10000
inner_learning = 0.1
gamma_interlayer = 1
top_k = 21
seed = np.random.seed()

new_DATA_MATRIX = np.random.rand(D_graph, DATA_MATRIX.shape[1])



class InterLayer:
    def __init__(self, D_graph, max_inner_dim, inter_dim=None, edge_threshold=0.02, gamma=1.0, seed=42):
        np.random.seed(seed)
        self.D_graph = D_graph
        self.edge_threshold = edge_threshold
        self.gamma = gamma
        self.inter_dim = inter_dim[0] if isinstance(inter_dim, list) else (inter_dim if inter_dim is not None else max_inner_dim[0] if isinstance(max_inner_dim, list) else max_inner_dim)
        self.max_input = 2 * (max_inner_dim[0] if isinstance(max_inner_dim, list) else max_inner_dim)
                    # Initialize weights proportional to synthetic correlation between nodes
        self.weights = {}
        self.bias = {}
        for i in range(D_graph):
            for j in range(D_graph):
                if i != j:
                    # small random + slight bias towards correlation
                    w_init = np.random.uniform(-0.1, 0.1, (self.inter_dim, self.max_input))
                    self.weights[(i,j)] = w_init
                    self.bias[(i,j)] = np.zeros(self.inter_dim)

    def compute_edge_activation(self, i, j, nested_reps):
        concat = np.concatenate([nested_reps[i], nested_reps[j]])
        concat = np.pad(concat, (0, max(0, self.max_input - len(concat))))[:self.max_input]

        # Normalize input to improve correlation
        concat = (concat - np.mean(concat)) / (np.std(concat) + 1e-12)

        # Compute activation
        v = self.weights[(i,j)].dot(concat) + self.bias[(i,j)]

        # Scale by correlation strength with input signals
        input_strength = np.clip(np.mean(np.abs(concat)), 0, 1)
        v = v * input_strength

        return 1 / (1 + np.exp(-v))

    def build_activations(self, Gmat, nested_reps):
        acts = {}
        for i in range(self.D_graph):
            for j in range(self.D_graph):
                if i == j:
                    continue
                if abs(Gmat[i,j]) > self.edge_threshold:
                    acts[(i,j)] = self.compute_edge_activation(i, j, nested_reps)
        return acts

    @staticmethod
    def pairwise_squared_corr(acts):
        if len(acts) < 2:
            return 0.0
        A = np.stack(list(acts.values()))
        A_centered = A - A.mean(axis=1, keepdims=True)
        stds = np.sqrt(np.sum(A_centered**2, axis=1) / (A.shape[1]-1) + 1e-12)
        cov = A_centered @ A_centered.T / (A.shape[1]-1)
        corr = cov / (np.outer(stds, stds) + 1e-12)
        np.fill_diagonal(corr, 0)
        return float((corr**2).sum())

    def mi_for_graph(self, Gmat, nested_reps):
        acts = self.build_activations(Gmat, nested_reps)
        if not acts:
            return 0.0
        return self.gamma * self.pairwise_squared_corr(acts)

    def correlate_shrink_interlayer(self, fmt_bounds=None, interaction_tensor=None, metrics_keys=None, verbose=True):
        """
        Compute Pearson correlation per node & metric between:
            - shrink factor (adaptive FMT)
            - mean outgoing inter-layer activations
        Returns: {node_idx: {metric: {'r':..., 'p':...}}}
        """
        from scipy.stats import pearsonr

        if metrics_keys is None:
            metrics_keys =self.MK

        D = self.D_graph

        # 1. Compute FMT bounds if not given
        if fmt_bounds is None:
            fmt_bounds = self.compute_fmt_with_bounds_adaptive(top_k=top_k)

        # 2. Get inter-layer activations if not provided
        if interaction_tensor is None:
            interaction_tensor = self.print_interactions(return_tensor=True, verbose=False)

        inter_mean = interaction_tensor.mean(axis=2)  # (D,D)
        shrink_factors = self.compute_fmt_shrink_factor(fmt_bounds, metrics_keys)  # (D, num_metrics)

        correlations = {}

        for i in range(D):
            correlations[i] = {}
            for k, key in enumerate(metrics_keys):
                # FMT shrink for node i (broadcasted across outgoing edges)
                shrink_vec = shrink_factors[i, k] * np.ones(D)
                # Outgoing inter-layer activations from node i
                inter_vec = inter_mean[i, :]
                # Remove self-loop
                mask = np.arange(D) != i
                shrink_vec = shrink_vec[mask]
                inter_vec = inter_vec[mask]

                # Compute Pearson correlation
                if np.std(inter_vec) > 1e-8:  # valid correlation
                    r, p = pearsonr(shrink_vec, inter_vec)
                else:
                    r, p = 0.0, 1.0  # no variability

                correlations[i][key] = {'r': r, 'p': p}
                if verbose:
                    print(f"Node {i} | {key} shrink vs inter-layer: r={r:.3f}, p={p:.3e}")

        return correlations


# ---------------- UNIFIED ACOR MULTIPLEX ----------------
class Fuzzy_Hierarchical_Multiplex:
    def __init__(self, candidate_dims, D_graph,
                 synthetic_targets, inner_learning,
                 gamma_interlayer=1.0, causal_flag=True,metrics=METRIC_KEYS,metric_mask=METRIC_TARGET):
        self.candidate_dims = candidate_dims
        self.D_graph = D_graph
        self.synthetic_targets = synthetic_targets
        self.inner_learning = inner_learning
        self.causal_flag = causal_flag
        self.best_dim_per_node = [len(t)-1 for t in synthetic_targets]  # last element as best dim
        self.MM = metric_mask
        self.MK = metrics
        self.MKI = metrics+['score']

        self.PLM = [[] for _ in range(self.D_graph)]
        self.PLMS = [[] for _ in range(self.D_graph)]
        self.nested_reps = [np.zeros(c[0]) for c in candidate_dims]
      #  self.best_dim_per_node = [candidate_dims[0] for _ in range(D_graph)]
        self.inter_layer = InterLayer(D_graph, max_inner_dim=max(candidate_dims), gamma=gamma_interlayer)
        self.chosen_Gmat = np.random.uniform(-0.5,0.5,(D_graph,D_graph))
        np.fill_diagonal(self.chosen_Gmat,0)
        self.l2_before, self.l2_after = [], []
        # ---------------- PER-NODE SIMPLE MLP ----------------
        # ---------------- PER-NODE MLP ----------------
        self.PLMS = [[] for _ in range(self.D_graph)]

        self.mlp_hidden = 16
        self.mlp_lr = 0.01
# Max target length across all nodes
        self.max_target_len = max(len(t['target']) for t in synthetic_targets)

        input_dim = len(self.MK) # <-- IMPORTANT (1,3 input)
        output_dim = self.max_target_len          # number of metrics

        self.node_mlps = []

        for node_idx in range(self.D_graph):
            input_dim = len(self.MK)  # number of input metrics
            output_dim = self.candidate_dims[node_idx][0]  # match node output
            mlp = {
                "W1": np.random.randn(input_dim, self.mlp_hidden) * 0.1,
                "b1": np.zeros(self.mlp_hidden),
                "W2": np.random.randn(self.mlp_hidden, output_dim) * 0.1,
                "b2": np.zeros(output_dim)
            }
            self.node_mlps.append(mlp)



    def mlp_forward(self, mlp, x):
        """
        x: (1,1)
        """

        z1 = x @ mlp["W1"] + mlp["b1"]
        h = np.tanh(z1)
        y = h @ mlp["W2"] + mlp["b2"]
        return y, h

    @staticmethod
    def mlp_forward(mlp, x):
        z1 = x @ mlp["W1"] + mlp["b1"]
        h = np.tanh(z1)
        y = h @ mlp["W2"] + mlp["b2"]
        return y, h



    def mlp_predict(mlp, weighted_input):
        x = np.array(weighted_input, dtype=float).reshape(1, -1)  # <-- correct
        y_pred, _ = Fuzzy_Hierarchical_Multiplex.mlp_forward(mlp, x)
        return y_pred.flatten()

    def mlp_train_step(self, mlp, x, y_true):
        """
        One SGD step with MSE loss
        x: (1,1)
        y_true: (1,M)
        """
        # Forward
        y_pred, h = self.mlp_forward(mlp, x)

        # Loss gradient
        dy = y_pred - y_true

        # Backprop
        dW2 = h.T @ dy
        db2 = dy[0]

        dh = dy @ mlp["W2"].T
        dz1 = dh * (1 - h**2)

        dW1 = x.T @ dz1
        db1 = dz1[0]

        # Update
        mlp["W2"] -= self.mlp_lr * dW2
        mlp["b2"] -= self.mlp_lr * db2
        mlp["W1"] -= self.mlp_lr * dW1
        mlp["b1"] -= self.mlp_lr * db1

        return float(np.mean((y_pred - y_true) ** 2))


    # ---------- INNER LOOP (FCM) ----------
    def run_inner(self, node_idx, target, D_fcm,
              steps=1000, lr_x=1, lr_y=0.01, lr_W=0.01,
              decorrelate_metrics=True):

        # --- Initialize activations ---
        x =np.random.uniform(-0.6, 0.6, D_fcm)# target.copy()
        y = np.random.uniform(-0.6, 0.6, D_fcm)

        # Pad target for L2 computation
        target_padded = np.pad(target, (0, len(self.nested_reps[node_idx]) - len(target)),
                            mode='constant', constant_values=0.5)
        self.l2_before.append(np.linalg.norm(self.nested_reps[node_idx] - target_padded))

        # --- FCM updates ---
        W = np.random.uniform(-0.6, 0.6, (D_fcm, D_fcm))
        np.fill_diagonal(W, 0)

        for _ in range(steps):
            z = y.dot(W) + x
            Theta_grad_z = z - target
            Theta_grad_x = Theta_grad_z
            Theta_grad_y = Theta_grad_z.dot(W.T)
            Theta_grad_W = np.outer(y, Theta_grad_z)

            x -= lr_x * np.clip(Theta_grad_x, -0.05, 0.05)
            y -= lr_y * np.clip(Theta_grad_y, -0.05, 0.05)
            W -= lr_W * np.clip(Theta_grad_W, -0.01, 0.01)

            x = np.clip(x, 0, 1)
            y = np.clip(y, 0, 1)
            np.fill_diagonal(W, 0)
            W = np.clip(W, -1, 1)

        # --- Pad FCM output to nested representation ---
        x_padded = np.pad(x, (0, len(self.nested_reps[node_idx]) - len(x)),
                        mode='constant', constant_values=0.5)
        self.nested_reps[node_idx] = x_padded
        self.l2_after.append(np.linalg.norm(x_padded - target_padded))

        # --- Extract node features ---
        metrics_evaluator = MetricsEvaluator(data_matrix=DATA_MATRIX)
        features = metrics_evaluator.extract_features(node_idx)
        feat_vals = np.array(list(features.values()))

        # --- Compute metrics scaled by activations + features ---
        metric_mask = METRIC_TARGET[node_idx]
        metric_values = {}

        for key, formula, mask in zip(METRIC_KEYS, METRIC_FORMULAS, metric_mask):
            if mask:
                # Match activations to features
                act_vals = x[:len(feat_vals)]
                # --- Inner weighted sum ---
                weighted_input = np.mean([1] * feat_vals)

                # --- Apply outer scale if available ---
                outer_scale = self.best_node_weights[node_idx] if hasattr(self, 'best_node_weights') else 1.0
                weighted_input *= outer_scale

                # Optional: small node bias for uniqueness
               # weighted_input += 0.05 * node_idx

                # Compute metric
                metric_values[key] = formula(weighted_input)
            else:
                metric_values[key] = 0.0


        # --- Total score ---
        metric_values['score'] = sum(metric_values.values())

        # --- Store inner activation for outer loop ---
      #  metric_values['x'] = x_padded.copy()        # store padded activation
       # metric_values['feat_vals'] = feat_vals.copy()  # store features# --- Build MLP training pair ---
        metric_output_vals = np.array(
            [v for k, v in metric_values.items()
            if k not in ['score', 'x', 'feat_vals']]
        )
        mlp = self.node_mlps[node_idx]

                # Input vector padded to MLP input size
        x_in_full = np.zeros(len(self.MK))
        x_in_full[:len(metric_output_vals)] = metric_output_vals
        x_in = x_in_full.reshape(1, -1)

        # Target vector padded or truncated to MLP output size
        output_dim = self.node_mlps[node_idx]["b2"].shape[0]
        y_out_full = np.zeros(output_dim)
        y_out_full[:min(len(feat_vals), output_dim)] = feat_vals[:output_dim]
        y_out = y_out_full.reshape(1, -1)

        mlp_loss = self.mlp_train_step(mlp, x_in, y_out)



        # --- Store PLMS trace (for diagnostics / replay) ---
        self.PLMS[node_idx].append(
            (float(weighted_input), metric_output_vals)
        )

        for i in range(self.D_graph):
            print(f"Node {i}, samples learned:", len(self.PLMS[i]))

        # --- Compute inter-layer MI ---
        mi_score = self.inter_layer.mi_for_graph(self.chosen_Gmat, self.nested_reps)

        return x, y, W, mi_score, metric_values





    def run_outer(self, outer_cost_limit=1000):
        node_metrics_list = self.capped_node_metrics  # Use cached metrics from run_inner
        raw_scores = np.array([m['score'] for m in node_metrics_list])

        # --- Apply cap if needed ---
        total_raw = raw_scores.sum()
        if total_raw > outer_cost_limit:
            scale_factor = outer_cost_limit / total_raw
            for metrics in node_metrics_list:
                for key in self.MKI:
                    metrics[key] *= scale_factor
            raw_scores *= scale_factor

        # --- Compute Fuzzy Metric Tensor ---
        fuzzy_tensor = self.compute_fuzzy_metric_tensor(normalize=False)
                # ✅ Persist weights / tensor so they are not lost
        self.weighted_fmt = fuzzy_tensor.copy()

        self.saved_nested_reps = [rep.copy() for rep in self.nested_reps]
        self.saved_Gmat = self.chosen_Gmat.copy()


        D = self.D_graph

        off_diag_mask = np.ones((D,D),dtype=bool)
        np.fill_diagonal(off_diag_mask,0)
        fmt_score_offdiag = fuzzy_tensor[off_diag_mask].sum()

        # --- Compute per-node contributions ---
        node_contributions = np.zeros(D)
        for i in range(D):
            own_score = raw_scores[i]
            fmt_contrib = fuzzy_tensor[i,:,:].sum() - fuzzy_tensor[i,i,:].sum()
            node_contributions[i] = own_score + self.inter_layer.gamma * fmt_contrib

        # --- Correlation penalty ---
        interaction_tensor = self.print_interactions(return_tensor=True, verbose=False)
        fmt_mean = fuzzy_tensor.mean(axis=2)
        inter_mean = interaction_tensor.mean(axis=2)
        corr_penalty = 0.0
        for i in range(D):
            fmt_vec = fmt_mean[i,:]
            inter_vec = inter_mean[i,:]
            if np.std(fmt_vec) > 1e-8 and np.std(inter_vec) > 1e-8:
                corr_penalty += abs(np.corrcoef(fmt_vec, inter_vec)[0,1])**2
        corr_penalty /= D

        combined_score = node_contributions.sum() - corr_penalty

        self.node_score_contributions = node_contributions
        self.correlation_penalty = corr_penalty

        return node_metrics_list, combined_score, node_contributions



    def run(self, outer_generations=outer_generations):
        best_score = -np.inf

        for gen in range(outer_generations):
            node_metrics_list = []

            for node_idx in range(self.D_graph):
                full_target = self.synthetic_targets[node_idx]['target']
                D_fcm = self.candidate_dims[node_idx][0]
                target = full_target[:D_fcm]

                _, _, _, _, metrics = self.run_inner(node_idx, target, D_fcm)
                node_metrics_list.append(metrics)

            # Outer loop
            self.capped_node_metrics = node_metrics_list
            _, capped_score, node_contributions = self.run_outer()

            if capped_score > best_score:
                best_score = capped_score
            else:
                print(f"\n--- Generation {gen} Metrics (NEW BEST) ---")
                for i, m in enumerate(node_metrics_list):
                    out_str = []
                    for k, v in m.items():
                        if np.isscalar(v):
                            out_str.append(f"{k}: {v:.2f}")
                        elif isinstance(v, np.ndarray):
                            pass
                            out_str.append(f"{k}: mean {v.mean():.2f}, shape {v.shape}")
                        else:
                            out_str.append(f"{k}: {v}")
                    print(f"Node {i} | " + " | ".join(out_str))

                print(f"\n--- Generation {gen} Node Contributions (NEW BEST) ---")
                for i, c in enumerate(node_contributions):
                    print(f"Node {i}: Contribution = {c:.4f}")

                print(f"Outer Score (capped): {capped_score:.3f} <-- NEW BEST")

        return best_score




    # ---------- VISUALIZATIONS ----------
    # ---------- VISUALIZATIONS ----------
    def plot_pointwise_minmax_elite(self, top_k=21):
        plt.figure(figsize=(14,3))
        for i in range(self.D_graph):
            # Node's actual dimension
            dim_i = self.candidate_dims[i][0]  # ✅ integer
            base = self.nested_reps[i][:dim_i]  # slice to candidate dim
            reps = np.clip(base + np.random.normal(0,0.05,(top_k,len(base))),0,1)
            y_min, y_max = reps.min(axis=0), reps.max(axis=0)
            y_sel = base

            # True target for this node, sliced to candidate dim
            y_true = self.synthetic_targets[i]['target'][:len(y_sel)]
            if len(y_true) < len(y_sel):
                y_true = np.pad(y_true, (0, len(y_sel)-len(y_true)), "constant")
            else:
                y_true = y_true[:len(y_sel)]

            plt.subplot(1,self.D_graph,i+1)
            plt.fill_between(range(len(y_min)),y_min,y_max,color='skyblue',alpha=0.4,label='Elite Interval')
            plt.plot(y_sel,'k-',lw=2,label='Estimated')
            plt.plot(y_true,'r--',lw=2,label='True')
            plt.ylim(0,1.05)
            plt.title(f"Node {i+1}")
            if i==0: plt.legend()
        plt.tight_layout()
        plt.show()


    def plot_nested_activations(self):
        plt.figure(figsize=(12,3))
        for i,rep in enumerate(self.nested_reps):
            dim_i = self.candidate_dims[i][0]
            rep_i = rep[:dim_i]  # slice to candidate dim
            plt.subplot(1,self.D_graph,i+1)
            plt.bar(range(len(rep_i)), rep_i, color=plt.cm.plasma(rep_i))
            plt.ylim(0,1)
            plt.title(f"Node {i+1}")
        plt.tight_layout()
        plt.show()


    def plot_outer_fuzzy_graph(self):
        G = nx.DiGraph()
        for i in range(self.D_graph): G.add_node(i)
        for i in range(self.D_graph):
            for j in range(self.D_graph):
                if i!=j and abs(self.chosen_Gmat[i,j])>0.02:
                    G.add_edge(i,j,weight=self.chosen_Gmat[i,j])
        node_sizes = [self.best_dim_per_node[i]*200 for i in range(self.D_graph)]
        edge_colors = ['green' if d['weight']>0 else 'red' for _,_,d in G.edges(data=True)]
        edge_widths = [abs(d['weight'])*3 for _,_,d in G.edges(data=True)]
        pos = nx.circular_layout(G)
        plt.figure(figsize=(6,6))
        nx.draw(G,pos,node_size=node_sizes,node_color='skyblue',
                edge_color=edge_colors,width=edge_widths,arrows=True,with_labels=True)
        plt.title("Outer Fuzzy Multiplex Graph")
        plt.show()
# ---------------- INTERACTIONS INSPECTOR ----------------

    def print_interactions(self, return_tensor=True, verbose=True):
            D_graph = self.D_graph
            inter_dim = self.inter_layer.inter_dim
            inter_tensor = np.zeros((D_graph, D_graph, inter_dim))

            acts = self.inter_layer.build_activations(self.chosen_Gmat, self.nested_reps)
            if not acts:
                if verbose:
                    print("No active edges above threshold.")
                return inter_tensor if return_tensor else None

            for (i, j), vec in acts.items():
                inter_tensor[i, j, :] = vec
                if verbose:
                    act_str = ", ".join([f"{v:.3f}" for v in vec])
                    print(f"Node {i} -> Node {j}: [{act_str}]")
            return inter_tensor if return_tensor else None

        # Move these outside of print_interactions (class-level)
    def print_l2_summary(self):
            print("\nL2 Distances to Target per Node:")
            for idx, (before, after) in enumerate(zip(self.l2_before, self.l2_after)):
                print(f"Node {idx}: Before={before:.4f}, After={after:.4f}")

    def compute_fuzzy_metric_tensor(self, normalize=True, verbose=False):
            """
            Computes a Fuzzy Metric Tensor (D_graph x D_graph x num_metrics)
            using current nested reps and node metrics.
            Each slice [i,j,:] represents metrics of node j (optionally weighted by Gmat[i,j])
            """
            metrics_keys =self.MK
            D = self.D_graph
            num_metrics = len(metrics_keys)
            tensor = np.zeros((D, D, num_metrics))

            metrics_evaluator = MetricsEvaluator(DATA_MATRIX)

            node_metrics = []
            for i, rep in enumerate(self.nested_reps):
                metrics = metrics_evaluator.compute_node_metrics(i, y=rep)
                node_metrics.append(np.array([metrics[k] for k in metrics_keys]))
            node_metrics = np.array(node_metrics)  # (D, num_metrics)

            for i in range(D):
                for j in range(D):
                    if i==j:
                        tensor[i,j,:] = node_metrics[j]
                    else:
                        weight = np.clip(abs(self.chosen_Gmat[i,j]), 0, 1)
                        tensor[i,j,:] = weight * node_metrics[j]

            if normalize:
                tensor = (tensor - tensor.min()) / (tensor.max() - tensor.min() + 1e-12)

            if verbose:
                print("Fuzzy Metric Tensor shape:", tensor.shape)

            return tensor



    def compute_fmt_shrink_factor(self, fmt_bounds, metrics_keys=None):
        """
        Returns shrink factor per node and metric.
        shrink_factor = 1 - (current_interval / original_interval)
        """
        if metrics_keys is None:
            metrics_keys = self.MK

        D = self.D_graph
        num_metrics = len(metrics_keys)
        shrink_factors = np.zeros((D, num_metrics))

        for i in range(D):
            for k in range(num_metrics):
                lower, upper = fmt_bounds[i, i, k, 0], fmt_bounds[i, i, k, 1]  # self-node interval
                interval_width = upper - lower + 1e-12  # normalized [0,1]
                shrink_factors[i, k] = 1 - interval_width  # more shrink = higher value

        return shrink_factors

    def compute_fmt_with_bounds_adaptive(self, top_k=21, max_shrink=0.5, metrics_keys=None):
        """
        Computes FMT bounds using pointwise min/max across elite solutions,
        and applies dynamic adaptive shrinking where variability is low.
        Returns tensor shape (D,D,num_metrics,2) [lower, upper].
        """
        if metrics_keys is None:
            metrics_keys =self.MK

        D = self.D_graph
        num_metrics = len(metrics_keys)
        tensor_bounds = np.zeros((D, D, num_metrics, 2))
        metrics_evaluator = MetricsEvaluator(DATA_MATRIX)

        variability = np.zeros((D, num_metrics))

        # Step 1: compute bounds from perturbations
        for i in range(D):
            base = self.nested_reps[i]
            reps = np.clip(base + np.random.normal(0, 0.05, (top_k, len(base))), 0, 1)
            metrics_matrix = np.zeros((top_k, num_metrics))
            for idx, rep in enumerate(reps):
                m = metrics_evaluator.compute_node_metrics(i, y=rep)
                metrics_matrix[idx, :] = [m[k] for k in metrics_keys]

            lower_i = metrics_matrix.min(axis=0)
            upper_i = metrics_matrix.max(axis=0)
            tensor_bounds[i, :, :, 0] = lower_i[np.newaxis, :]  # broadcast to all j
            tensor_bounds[i, :, :, 1] = upper_i[np.newaxis, :]
            variability[i, :] = metrics_matrix.std(axis=0)

        # Step 2: adaptive shrinking
        for i in range(D):
            for j in range(D):
                for k in range(num_metrics):
                    lower, upper = tensor_bounds[i,j,k,0], tensor_bounds[i,j,k,1]
                    mean = (lower + upper)/2
                    var_norm = min(1.0, variability[i,k]/(upper-lower + 1e-12))
                    shrink_factor = max_shrink * (1 - var_norm)
                    tensor_bounds[i,j,k,0] = mean - shrink_factor*(mean - lower)
                    tensor_bounds[i,j,k,1] = mean + shrink_factor*(upper - mean)

        return tensor_bounds



    def plot_fuzzy_metric_tensor_heatmaps(self, fuzzy_tensor=None, metrics_keys=METRIC_KEYS):
        """
        Plot a heatmap panel for each metric in the FMT.
        Rows: source node i
        Columns: target node j
        """
        if fuzzy_tensor is None:
            fuzzy_tensor = self.compute_fuzzy_metric_tensor(normalize=True)

        D = self.D_graph
        num_metrics = len(metrics_keys)

        fig, axes = plt.subplots(1, num_metrics, figsize=(4*num_metrics,4))
        if num_metrics == 1: axes = [axes]

        for k, key in enumerate(metrics_keys):
            data = fuzzy_tensor[:,:,k]
            im = axes[k].imshow(data, cmap='viridis', vmin=0, vmax=1)
            for i in range(D):
                for j in range(D):
                    axes[k].text(j,i,f"{data[i,j]:.2f}",ha='center',va='center',color='white',fontsize=9)
            axes[k].set_xticks(range(D))
            axes[k].set_yticks(range(D))
            axes[k].set_xticklabels([f'Node {j}' for j in range(D)])
            axes[k].set_yticklabels([f'Node {i}' for i in range(D)])
            axes[k].set_title(f'FMT - {key}')

        fig.colorbar(im, ax=axes, orientation='vertical', fraction=0.025, pad=0.04, label='Normalized Metric Value')
        plt.tight_layout()
        plt.show()

    def compute_fmt_with_elite_bounds(self, top_k=21):
        """
        Computes FMT bounds using pointwise min/max across elite solutions.
        Returns tensor shape (D,D,num_metrics,2) [lower, upper].
        """
        metrics_keys = self.MK
        D = self.D_graph
        num_metrics = len(metrics_keys)
        tensor_bounds = np.zeros((D,D,num_metrics,2))

        metrics_evaluator = MetricsEvaluator(DATA_MATRIX)

        for i in range(D):
            # Generate top_k perturbations around current nested_rep (like in plot_pointwise_minmax_elite)
            base = self.nested_reps[i]
            reps = np.clip(base + np.random.normal(0,0.05,(top_k,len(base))),0,1)

            # Compute node metrics for each perturbed solution
            metrics_matrix = np.zeros((top_k, num_metrics))
            for idx, rep in enumerate(reps):
                m = metrics_evaluator.compute_node_metrics(i, y=rep)
                metrics_matrix[idx,:] = [m[k] for k in metrics_keys]

            # Compute pointwise min/max across elite solutions
            lower_i = metrics_matrix.min(axis=0)
            upper_i = metrics_matrix.max(axis=0)

            # Fill bounds tensor for all source nodes (i->j)
            for j in range(D):
                tensor_bounds[i,j,:,0] = lower_i
                tensor_bounds[i,j,:,1] = upper_i

        return tensor_bounds


    def plot_fmt_with_bounds(self, fmt_tensor_bounds):
        """
        Plot the FMT with lower/upper bounds, applying outer-loop weights if available.
        Single row: mean across all source nodes
        Columns: metrics (mean across target nodes)
        Colors reflect actual weighted values (no normalization).
        """
        import matplotlib.pyplot as plt
        import numpy as np

        D = self.D_graph
        metrics_keys = self.MK
        M_actual = len(metrics_keys)

        # Compute mean value per metric across target nodes
        mean_vals = (fmt_tensor_bounds[:, :, :, 0] + fmt_tensor_bounds[:, :, :, 1]) / 2  # shape (D, D, M_actual)
        mean_vals = mean_vals.mean(axis=1)  # mean across targets -> (D, M_actual)

        # Take mean across nodes to reduce to 1xM
        mean_vals = mean_vals.mean(axis=0, keepdims=True)  # shape (1, M_actual)

        # Apply outer-loop weights if available
        if hasattr(self, 'best_alpha') and hasattr(self, 'best_w_contrib'):
            mean_weight = (self.best_alpha * self.best_w_contrib).mean()
            mean_vals = mean_vals * mean_weight

        # Plot actual weighted FMT values (no per-column normalization)
        fig, ax = plt.subplots(figsize=(1.2*M_actual + 4, 2))
        im = ax.imshow(mean_vals, cmap='viridis', aspect='auto')

        # Annotate cells with dynamic text color
        vmin, vmax = mean_vals.min(), mean_vals.max()
        for i in range(mean_vals.shape[0]):  # only 1 row
            for k in range(M_actual):
                val = mean_vals[i, k]
                color = 'white' if val < (vmin + 0.5*(vmax - vmin)) else 'black'
                ax.text(k, i, f"{val:.2f}", ha='center', va='center', color=color, fontsize=8)

        ax.set_xticks(range(M_actual))
        ax.set_xticklabels(metrics_keys[:M_actual], rotation=45, ha='right')
        ax.set_yticks([0])
        ax.set_yticklabels(['Mean across nodes'])
        ax.set_title("Weighted FMT with Bounds (Collapsed to 1 Row)")
        fig.colorbar(im, ax=ax, label='Weighted Mean Metric Value')
        plt.tight_layout()
        plt.show()


    def plot_node_score_contribution(self, metrics_keys=METRIC_KEYS):
        """
        Plot per-node total score contribution:
            - 3 panels: Raw, FMT (interaction), Total
            - Diagonal represents raw contributions
            - FMT scaled by best_weights if available
            - Annotated cells
            - Normalized across metrics for consistent visualization
        """
        D = self.D_graph
        node_contributions = np.array(self.node_score_contributions)

        # --- FMT contribution ---
        if hasattr(self, 'weighted_fmt'):
            fuzzy_tensor = np.array(self.weighted_fmt)
        else:
            fuzzy_tensor = self.compute_fuzzy_metric_tensor(normalize=True)

        # Normalize across entire tensor for plotting
        fuzzy_tensor_norm = (fuzzy_tensor - fuzzy_tensor.min()) / (fuzzy_tensor.max() - fuzzy_tensor.min() + 1e-12)
        fmt_matrix = fuzzy_tensor_norm.sum(axis=2)  # sum over metrics
        np.fill_diagonal(fmt_matrix, 0)

        # --- Raw matrix on diagonal ---
        raw_matrix = np.zeros((D,D))
        np.fill_diagonal(raw_matrix, node_contributions)

        # --- Total contribution ---
        total_matrix = raw_matrix + fmt_matrix

        # --- Global min/max for color scale ---
        global_min, global_max = total_matrix.min(), total_matrix.max()

        # --- Plot ---
        fig, axes = plt.subplots(1, 3, figsize=(15, 4))
        matrices = [raw_matrix, fmt_matrix, total_matrix]
        titles = ["Raw Node Contribution", "Normalized FMT Contribution", "Total Contribution"]

        for ax, mat, title in zip(axes, matrices, titles):
            im = ax.imshow(mat, cmap='viridis', vmin=0, vmax=1)  # normalized
            for i in range(D):
                for j in range(D):
                    ax.text(j, i, f"{mat[i,j]:.2f}", ha='center', va='center', color='white', fontsize=8)
            ax.set_title(title)
            ax.set_xticks(range(D))
            ax.set_xticklabels([f"Node {i+1}" for i in range(D)])
            ax.set_yticks(range(D))
            ax.set_yticklabels([f"Node {i+1}" for i in range(D)])

        fig.colorbar(im, ax=axes, orientation='vertical', fraction=0.025, pad=0.04, label='Contribution Value')
        plt.tight_layout()
        plt.show()

    def correlate_fmt_interactions_per_node(self, fmt_bounds=None, interaction_tensor=None, verbose=True):
        """
        Correlate the FMT bounds with inter-layer interactions per node and per metric.
        Returns a dict of shape: {node_idx: {metric: {'r':..., 'p':...}}}.
        """
        from scipy.stats import pearsonr
        import matplotlib.pyplot as plt

        metrics_keys = self.MK
        D = self.D_graph

        # Compute tensors if not provided
        if fmt_bounds is None:
            fmt_bounds = self.compute_fmt_with_elite_bounds(top_k=21)
        if interaction_tensor is None:
            interaction_tensor = self.print_interactions(return_tensor=True, verbose=False)

        # Reduce interaction tensor along inter_dim
        inter_mean = interaction_tensor.mean(axis=2)  # (D,D)

        node_correlations = {}

        for i in range(D):
            node_correlations[i] = {}
            for k, key in enumerate(metrics_keys):
                # FMT bounds for target node j from source i (mean of lower/upper)
                fmt_mean = fmt_bounds[i,:,k,:].mean(axis=1)  # shape (D,)
                # Interaction tensor for edges from node i to j
                inter_vec = inter_mean[i,:]  # shape (D,)
                # Pearson correlation
                corr, pval = pearsonr(fmt_mean, inter_vec)
                node_correlations[i][key] = {'r': corr, 'p': pval}

                if verbose:
                    print(f"Node {i} | {key}: r = {corr:.3f}, p = {pval:.3e}")
                    plt.figure(figsize=(4,3))
                    plt.scatter(fmt_mean, inter_vec, alpha=0.7, edgecolor='k', color='skyblue')
                    plt.xlabel(f"FMT {key} (Node {i} -> others)")
                    plt.ylabel(f"Interaction mean (Node {i} -> others)")
                    plt.title(f"Node {i} | {key} correlation: r={corr:.3f}")
                    plt.grid(True)
                    plt.show()

        return node_correlations

    def plot_fmt_with_run_metrics(self, metrics_keys=None):
        """
        Plot FMT heatmaps using the actual metrics from the last run_outer.
        Rows: source nodes
        Columns: metrics (mean across target nodes)
        """
        import matplotlib.pyplot as plt
        import numpy as np

        if metrics_keys is None:
            metrics_keys = self.MK

        D = self.D_graph
        M_actual = len(metrics_keys)

        if not hasattr(self, 'capped_node_metrics'):
            raise ValueError("No node metrics available. Run run_outer() first.")

        # Build FMT from actual node metrics
        weighted_fmt = np.zeros((D, D, M_actual))
        for i in range(D):
            for j in range(D):
                for k, key in enumerate(metrics_keys):
                    val = self.capped_node_metrics[j][key]  # take actual metric of node j
                    # Apply Gmat weight for off-diagonal
                    if i != j:
                        val *= np.clip(abs(self.chosen_Gmat[i,j]), 0, 1)
                    weighted_fmt[i,j,k] = val#*self.D_graph*

        # Mask metrics if needed
        for i in range(D):
            for k in range(M_actual):
                if not METRIC_TARGET[i][k]:
                    weighted_fmt[i, :, k] = 0.0

        # Average across target nodes
        mean_vals = weighted_fmt.mean(axis=1)  # (D, M_actual)

        # Plot
        fig, ax = plt.subplots(figsize=(1.2*M_actual + 4, 0.35*D + 4))
        im = ax.imshow(mean_vals, cmap='viridis', aspect='auto')

        vmin, vmax = mean_vals.min(), mean_vals.max()
        for i in range(D):
            for k in range(M_actual):
                val = mean_vals[i, k]
                color = 'white' if val < (vmin + 0.5*(vmax - vmin)) else 'black'
                ax.text(k, i, f"{val:.2f}", ha='center', va='center', color=color, fontsize=8)

        ax.set_xticks(range(M_actual))
        ax.set_xticklabels(metrics_keys[:M_actual], rotation=45, ha='right')
        ax.set_yticks(range(D))
        ax.set_yticklabels([f"Node {i}" for i in range(D)])
        ax.set_title("Weighted FMT Metrics (Actual Run Output)")
        fig.colorbar(im, ax=ax, label='Weighted Metric Value')
        plt.tight_layout()
        plt.show()


    def correlation_penalty(self, fmt_bounds=None, interaction_tensor=None):
        """
        Computes a penalty term that is high if per-node FMT metrics correlate with interactions.
        Returns total penalty to subtract from the outer score.
        """
        from scipy.stats import pearsonr

        D = self.D_graph
        metrics_keys = self.MK
        if fmt_bounds is None:
            fmt_bounds = self.compute_fmt_with_elite_bounds(top_k=top_k)
        if interaction_tensor is None:
            interaction_tensor = self.print_interactions(return_tensor=True, verbose=False)

        inter_mean = interaction_tensor.mean(axis=2)
        total_penalty = 0.0

        for i in range(D):
            for k in range(len(metrics_keys)):
                fmt_mean = fmt_bounds[i,:,k,:].mean(axis=1)
                inter_vec = inter_mean[i,:]
                if np.std(fmt_mean) > 1e-8 and np.std(inter_vec) > 1e-8:
                    corr, _ = pearsonr(fmt_mean, inter_vec)
                    total_penalty += abs(corr)  # penalize high correlation

        # normalize by number of nodes × metrics
        total_penalty /= (D * len(metrics_keys))**2
        return total_penalty


# ---------------- USAGE ----------------
if __name__ == "__main__":
    optimizer = Fuzzy_Hierarchical_Multiplex(
        candidate_dims, D_graph,
        synthetic_targets,
        inner_learning, gamma_interlayer=0,
        causal_flag=False
    )
    metrics_list = optimizer.run()
    optimizer.plot_pointwise_minmax_elite()
    optimizer.plot_nested_activations()
    # Compute FMT with elite bounds
    fmt_elite_bounds = optimizer.compute_fmt_with_elite_bounds(top_k=top_k+10)

# Plot as heatmaps
    optimizer.plot_fmt_with_run_metrics()

    # Compute fuzzy multiplex tensor
    fmt_tensor = optimizer.compute_fuzzy_metric_tensor(normalize=False)
    optimizer.plot_fuzzy_metric_tensor_heatmaps(fmt_tensor)

    # Compute FMT with bounds (minimax elite intervals)
    optimizer.plot_node_score_contribution()
    optimizer.plot_outer_fuzzy_graph()
  #  optimizer.print_interactions()
    tensor = optimizer.print_interactions()

    print("Tensor shape:", tensor.shape,'\n',tensor)
    # Compute tensors first

    #fmt_elite_bounds = optimizer.compute_fmt_with_elite_bounds(top_k=top_k)
    #interaction_tensor = optimizer.print_interactions(return_tensor=True, verbose=False)
    #optimizer.plot_fmt_with_bounds(fmt_elite_bounds)

    #interaction_tensor = optimizer.print_interactions(return_tensor=True, verbose=False)

    # Get per-node, per-metric correlations
    #node_metric_corrs = optimizer.correlate_fmt_interactions_per_node(
     #   fmt_bounds=fmt_elite_bounds,
      #  interaction_tensor=interaction_tensor
   # )

__Estimates Target Activavtion Based on CRN desired metric numbers__

In [ ]:
import numpy as np

# Choose a node and an input
node_idx = 2
weighted_input = np.array([0.85, 0.3, 0.2])


# Grab that node's trained MLP
mlp = optimizer.node_mlps[node_idx]

# Use the class helper directly (NO self)
predicted_metrics = Fuzzy_Hierarchical_Multiplex.mlp_predict(
    mlp,
    weighted_input
)

print(f"Node {node_idx}")
print("Input weighted_input:", weighted_input)
print("Predicted metrics:", np.round(predicted_metrics, 4))


In [ ]:
# --- Generate fresh synthetic targets ---
synthetic_targets_new= synthetic_targets
# --- Minimal FHM + FCM run with new targets ---
optimizer = Fuzzy_Hierarchical_Multiplex(
    candidate_dims=candidate_dims,
    D_graph=D_graph,
    synthetic_targets=synthetic_targets_new,
    inner_learning=inner_learning,
    gamma_interlayer=0.0,
    causal_flag=False
)

# Store metrics during first run
node_metrics_list = []

for node_idx in range(D_graph):
    full_target = synthetic_targets_new[node_idx]['target']
    D_fcm = candidate_dims[node_idx][0]  # candidate dimension
    target = full_target[:D_fcm]        # slice target to node FCM size

    # Run inner FCM loop
    x, y, W, mi_score, metrics = optimizer.run_inner(node_idx, target, D_fcm)
    node_metrics_list.append(metrics)

    #print(f"\nNode {node_idx} FHM output activations:\n{x}\n")
    print(f"Node {node_idx} metrics:\n{metrics}\n")

# Compute scores and CRM tiers
scores = [m['score'] for m in node_metrics_list]
scores_norm = (np.array(scores) - min(scores)) / (max(scores) - min(scores) + 1e-12)

for node_idx, score_norm in enumerate(scores_norm):
    if score_norm > 0.66:
        tier = 'High'
    elif score_norm > 0.33:
        tier = 'Medium'
    else:
        tier = 'Low'
    print(f"Node {node_idx}: CRM Tier = {tier}, Normalized Score = {score_norm:.3f}")


# FCM model

In [ ]:
import numpy as np

class FCMWithTargets:
    def __init__(self, synthetic_targets_new, seed=42):
        """
        synthetic_targets: list of dicts with 'target' key
        """
        np.random.seed(seed)
        self.synthetic_targets = synthetic_targets
        self.num_nodes = len(synthetic_targets)
        self.max_dim = max(len(t['target']) for t in synthetic_targets)

        # Initialize FCM weight matrices per node (square per node dim)
        self.W = [np.random.uniform(-0.5, 0.5, (len(t['target']), len(t['target'])))
                  for t in synthetic_targets]
        for w in self.W:
            np.fill_diagonal(w, 0)

        # Node activations
        self.activations = [np.zeros(len(t['target'])) for t in synthetic_targets]

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def run(self, steps=50, learning_rate=0.1):
        """
        Run FCM updates towards targets
        """
        for node_idx, t_dict in enumerate(self.synthetic_targets):
            target = t_dict['target']
            x = self.activations[node_idx]
            W = self.W[node_idx]

            for _ in range(steps):
                input_signal = W.dot(x) + learning_rate * (target - x)
                x = self.sigmoid(input_signal)

            self.activations[node_idx] = x
        return self.activations

    def compute_node_scores(self):
        """
        Example metric: average deviation from target
        Higher score = closer to target
        """
        scores = []
        for node_idx, t_dict in enumerate(self.synthetic_targets):
            target = t_dict['target']
            x = self.activations[node_idx]
            # score = inverse of L2 distance to target
            score = 1 / (1 + np.linalg.norm(x - target))
            scores.append(score)
        return scores

    def map_crm_tiers(self, scores, thresholds=(0.3, 0.6)):
        """
        Map node scores to CRM tiers
        """
        tiers = []
        for s in scores:
            if s > thresholds[1]:
                tier = 'High'
            elif s > thresholds[0]:
                tier = 'Medium'
            else:
                tier = 'Low'
            tiers.append(tier)
        return tiers

# ---------------- USAGE ----------------

fcm = FCMWithTargets(synthetic_targets_new)
activations = fcm.run(steps=100, learning_rate=0.2)

scores = fcm.compute_node_scores()
tiers = fcm.map_crm_tiers(scores)

for i, (act, score, tier) in enumerate(zip(activations, scores, tiers)):
    print(f"Node {i} | Activations: {act} | \nScore: {score:.3f} | CRM Tier: {tier}\n")


In [ ]:
import numpy as np

num_folds = 20
D_graph = len(candidate_dims)  # number of nodes
results_summary = []

# --- Customer reference target simulation ---
def generate_customer_targets(D_graph, candidate_dims):
    synthetic_targets = []
    for i in range(D_graph):
        dim = candidate_dims[i][0]
        base = np.random.rand(dim)
        if i % 3 == 0:
            target = 0.7 + 0.3 * base  # High preference
        elif i % 3 == 1:
            target = 0.3 + 0.4 * base  # Medium
        else:
            target = 0.0 + 0.3 * base  # Low
        synthetic_targets.append({'target': target})
    return synthetic_targets

# --- FCM baseline class ---
class FCMWithTargets:
    def __init__(self, synthetic_targets):
        self.synthetic_targets = synthetic_targets
        self.num_nodes = len(synthetic_targets)
        self.W = [np.random.uniform(-0.5, 0.5, (len(t['target']), len(t['target']))) for t in synthetic_targets]
        for w in self.W: np.fill_diagonal(w, 0)
        self.activations = [np.zeros(len(t['target'])) for t in synthetic_targets]

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def run(self, steps=100, learning_rate=0.2):
        for node_idx, t_dict in enumerate(self.synthetic_targets):
            target = t_dict['target']
            x = self.activations[node_idx]
            W = self.W[node_idx]
            for _ in range(steps):
                x = self.sigmoid(W.dot(x) + learning_rate*(target - x))
            self.activations[node_idx] = x
        return self.activations

# --- Run comparison over folds ---
for fold in range(num_folds):
    synthetic_targets_new = generate_customer_targets(D_graph, candidate_dims)

    # --- Minimal FHM run ---
    optimizer = Fuzzy_Hierarchical_Multiplex(
        candidate_dims=candidate_dims,
        D_graph=D_graph,
        synthetic_targets=synthetic_targets_new,
        inner_learning=inner_learning,
        gamma_interlayer=0.0,
        causal_flag=False
    )

    node_metrics_list = []
    for node_idx in range(D_graph):
        full_target = synthetic_targets_new[node_idx]['target']
        D_fcm = candidate_dims[node_idx][0]
        target = full_target[:D_fcm]
        _, _, _, _, metrics = optimizer.run_inner(node_idx, target, D_fcm)
        node_metrics_list.append(metrics)

    scores_fhm = np.array([m['score'] for m in node_metrics_list])
    # Normalize to 0-1 for CRM tiers
    norm_fhm = (scores_fhm - scores_fhm.min()) / (scores_fhm.max() - scores_fhm.min() + 1e-12)
    tiers_fhm = ['High' if s>0.66 else 'Medium' if s>0.33 else 'Low' for s in norm_fhm]

    # --- FCM baseline run ---
    fcm = FCMWithTargets(synthetic_targets_new)
    activations = fcm.run()

    # --- Compute FCM node metrics using MetricsEvaluator with padded targets ---
    max_len = max(len(t['target']) for t in synthetic_targets_new)
    data_matrix = np.array([np.pad(t['target'], (0, max_len - len(t['target'])), 'constant')
                            for t in synthetic_targets_new])

    evaluator = MetricsEvaluator(
        data_matrix=data_matrix,
        feature_target=FEATURE_TARGET[:D_graph],
        metric_target=METRIC_TARGET[:D_graph]
    )

    scores_fcm = np.array([evaluator.compute_node_metrics(i)['score'] for i in range(D_graph)])
    norm_fcm = (scores_fcm - scores_fcm.min()) / (scores_fcm.max() - scores_fcm.min() + 1e-12)
    tiers_fcm = ['High' if s>0.66 else 'Medium' if s>0.33 else 'Low' for s in norm_fcm]

    # --- Store fold summary ---
    results_summary.append({
        'fold': fold+1,
        'FHM_mean': scores_fhm.mean(),
        'FHM_std': scores_fhm.std(),
        'FHM_tiers': {'High': tiers_fhm.count('High'), 'Medium': tiers_fhm.count('Medium'), 'Low': tiers_fhm.count('Low')},
        'FCM_mean': scores_fcm.mean(),
        'FCM_std': scores_fcm.std(),
        'FCM_tiers': {'High': tiers_fcm.count('High'), 'Medium': tiers_fcm.count('Medium'), 'Low': tiers_fcm.count('Low')}
    })

# --- Print concise summary ---
for res in results_summary:
    print(f"Fold {res['fold']} | "
          f"FHM Mean: {res['FHM_mean']:.3f}, Std: {res['FHM_std']:.3f}, Tiers: {res['FHM_tiers']} | "
          f"FCM Mean: {res['FCM_mean']:.3f}, Std: {res['FCM_std']:.3f}, Tiers: {res['FCM_tiers']}")


# Techincal Report

__Abstract__

Here is a comparison between an FCM and the Fuzzy_Hierarchical_Multiplex with respect to a micro grid problem and four processes; the program uses user based inputs to determine service metrics and compatibility. In addition the CRM node-process evaluation is done using an FHM and FCM to. From the analysis it is determined that the FHM model is more flexible and efficient than an FCM in regards to assigning the better node/process, that is the process with desired attributes, to the customer.



__Data Generator__
In this script synthetic data that simulates a micro grid is generated and has four distinct patterns. The energy grid has four processes as a service let s say and so there each one has different attributes and priorities each time. To clarify, the data is the random, however, the way that the data is evaluated is directly stemming from the attributes that the user assigned. More specifically, in node one, attributes 1 to 5 are used to evaluate random data, and NOT generate it, etc. This way, every time the data is unrelated to the evaluation.



__Metrics Evaluator__
To determine the fitness of the targets an object is called which evaluates each given objective for the given data, or the overall node objectives that matter for each node, per node. To accomplish that, both the objective functions as well as the objectives are user inputs; furthermore, the objective evaluation is done using an exact method. To expound, the metrics are derived solving an optimization problem each time for each node. In this example Branch and bound was used to find best metrics per node.

Note: (There are feature masks, but are not reflected on the plots and are reflected on the prints only)


__FHM__
Now the FHM model is used and it optimizes several things, firstly it accepts user inputs for evaluation and models each target process accordingly. It learns the weights while using metrics evaluator to optimize the metrics and then assignes the metrics to the respective activations. It has learned to transcend from activations to metrics this way. Then, it uses the outer objecitves and optimize node contributions to relearn the metrics from the scores and eventually. Now both activations and outer objectives can be used to calculate map the metric. Secondly, given input data from the data metrics, it analyzes the system services and provides an overall metric patron for the targets. Lastly, when a random input is the model can rerun it s formulas to calculate per node fitness and provide a normalized score.

__FCM__
The FCM tries to learn the targets and provide a reference for any relevant data. The targets are new data generated and the FCM is trained on this, it doesnt have a prior knowledge. It loses the comparison which will be explain after that. it has to be noted that this FCM does not learn the data so well.

__Interpretation__
The FHM has the ability to model data given user inputs which makes it a very strong model for deduction. On the other hand the FCM is data based; moreover, regardles off the activations the data fit is better on the FHM as it was tuned this way. Here it is important to distinguish between DATA and METRICS. While FHM maps the metrics the results only reflect that: __the data we want is provided for several attributes that one has assigned and nothing more.__ It is only the implications and no correlation, however, provided that the FHM mapped the data in a specific manner there is a score for each instance.

__Remarks__
The system has been used to model data and to determine outputs or more specifficaly service scores based on process. On the other hand the FCM uses raw data per node to calculate mean activations, which is a baseline. To attest, one can simply learn the data or maximize activations and there is another baseline with different score. Finally, the model is flexible enough, yet requires  some intervention. I think it works !!!



# Node Features and Metric Mapping
__based on chat translation__

Each node \(i\) has a feature vector:

$$
\mathbf{f}_i = [f_{i1}, f_{i2}, \dots, f_{iF}] \in [0,1]^F
$$

with a feature mask \(M^{(f)}_i \in \{0,1\}^F\), so that the active features are:

$$
\tilde{\mathbf{f}}_i = \mathbf{f}_i \odot M^{(f)}_i
$$

For metric \(k\) with formula \(g_k(\cdot)\) and relevant feature set \(S_k\):

$$
x_{ik} = \frac{1}{|S_k \cap \text{active}(i)|} \sum_{j \in S_k \cap \text{active}(i)} f_{ij}
$$

$$
m_{ik} = g_k(x_{ik})
$$

Node total score:

$$
\text{score}_i = \sum_{k=1}^{K} m_{ik}
$$

---

# Inner Loop: Fuzzy Cognitive Map (FCM)

Each node \(i\) has inner activation vectors:

$$
\mathbf{x}_i \in [0,1]^{D_i}, \quad \mathbf{y}_i \in [0,1]^{D_i}
$$

and FCM weight matrix:

$$
W_i \in \mathbb{R}^{D_i \times D_i}, \quad W_{ii} = 0
$$

**FCM updates per step \(t\):**

$$
\mathbf{z}_i^{(t)} = \mathbf{x}_i^{(t)} + W_i^{(t)} \mathbf{y}_i^{(t)}
$$

$$
\Theta^{(t)}_{\mathbf{x}} = \mathbf{z}_i^{(t)} - \mathbf{x}_i^\text{target}, \quad
\Theta^{(t)}_{\mathbf{y}} = \Theta^{(t)}_{\mathbf{x}} \cdot (W_i^{(t)})^T
$$

$$
\Theta^{(t)}_{W} = \mathbf{y}_i^{(t)} \otimes \Theta^{(t)}_{\mathbf{x}}
$$

**Gradient updates:**

$$
\mathbf{x}_i^{(t+1)} = \mathbf{x}_i^{(t)} - \eta_x \cdot \text{clip}(\Theta^{(t)}_{\mathbf{x}}, -\delta_x, \delta_x)
$$

$$
\mathbf{y}_i^{(t+1)} = \mathbf{y}_i^{(t)} - \eta_y \cdot \text{clip}(\Theta^{(t)}_{\mathbf{y}}, -\delta_y, \delta_y)
$$

$$
W_i^{(t+1)} = \text{clip}(W_i^{(t)} - \eta_W \cdot \text{clip}(\Theta^{(t)}_{W}, -\delta_W, \delta_W), -1, 1)
$$

---

# Inter-layer Activations

Let the graph be \(G\) with adjacency \(G_{ij}\). The inter-layer activation from node \(i\) to \(j\) is:

$$
\mathbf{a}_{ij} = \sigma \Big( W_{ij} \cdot [\mathbf{x}_i \| \mathbf{x}_j] + \mathbf{b}_{ij} \Big) \cdot \text{strength}(\mathbf{x}_i, \mathbf{x}_j)
$$

where \(\sigma\) is the sigmoid function and \(\| \) denotes concatenation.

Mutual information / correlation across edges:

$$
MI(G) = \gamma \sum_{(i,j) \in E} \text{corr}^2(\mathbf{a}_{ij}, \mathbf{a}_{ji})
$$

---

# Fuzzy Metric Tensor (FMT)

Define FMT tensor:

$$
FMT_{i,j,k} =
\begin{cases}
m_{jk}, & i = j \\
|G_{ij}| \cdot m_{jk}, & i \neq j
\end{cases}
$$

Normalized across metrics:

$$
\hat{FMT}_{i,j,k} = \frac{FMT_{i,j,k} - \min(FMT)}{\max(FMT) - \min(FMT) + \epsilon}
$$

---

# Outer Loop: Node Contribution

Node contributions with outer weights \(alpha_i\):

$$
C_i = \alpha_i \left( \text{raw_score}_i + \gamma \sum_{j \neq i} \hat{FMT}_{i,j,:} \right)
$$

Outer objective (maximize total contribution minus correlation penalty):

$$
\text{Score}_\text{outer} = \sum_i C_i - \frac{1}{D \cdot K} \sum_i \sum_k |\text{corr}(FMT_{i,:,k}, A_{i,:})|
$$
